# Example notebook for WeLoop chatbot based on local Falcon-7B

### Install prerequisites:

In [1]:
!pip install --upgrade pip
!pip install -r requirements.txt

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.299
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
!nvidia-smi

Fri Sep 22 18:05:56 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A4500               On  | 00000000:82:00.0 Off |                  Off |
| 30%   29C    P8              14W / 200W |      2MiB / 20470MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Preprocess text files:

In [4]:
import gc
import torch
from typing import List
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

In [5]:
# load and process the text files

loader = DirectoryLoader('./data/text', glob="*.txt", loader_cls=TextLoader)
documents = loader.load()
len(documents)

17

In [6]:
# splitting text into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

32

In [7]:
# text example

texts[22]

Document(page_content='VERSION: 2.31.2\ncreated: 28 mars 23\n\n\nTitre: Les nouveautés de Mars 2023 / Notifications Public Mode !\n\n-NEW ✨\nNotification Pop Up - Public Mode\n\n\nGrande avancée coté produit !\n\nIl est désormais possible d\'afficher la release note sous forme de Pop Up au-dessus du Wigdet pour les utilisateurs en mode public. Ainsi, quand un admin publie une Release Note elle s\'affichera maintenant auprès de tous les utilisateurs disposant du Widget.\n\nIl est également possible d\'agrandir à présent les images pour pouvoir mieux visualiser leur contenu !\n\n\nDe plus, un bouton "Traduire" est à présent affiché dans la Pop Up afin de permettre aux utilisateurs de traduire le texte affiché dans une Release Note. En tant qu\'Admin vous vous assurez donc que tous vos utilisateurs pourront prendre connaissance de votre Release Note, peu importe la langue dans laquelle vous l\'avez rédigé !\n\n\n\n- Analytics', metadata={'source': 'data/text/Les nouveautés de Mars 2023 _

In [8]:
# set Embedding model from HuggingFace

model_name = "intfloat/e5-large-v2"

hf = HuggingFaceEmbeddings(model_name=model_name)

In [9]:
# embed and store the texts
# supplying a persist_directory will store the embeddings on disk
persist_directory = 'data/db'

## here is the new embeddings being used
embedding = hf #instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [10]:
# set retriever

retriever = vectordb.as_retriever()

In [11]:
# test retriever

docs = retriever.get_relevant_documents("WeLoop est quoi ?")
len(docs)

4

In [12]:
# document example

docs[0]

Document(page_content="Titre : WeLoop c'est quoi ?\n\nDescription: WeLoop est une plateforme Saas basée sur la communauté et l'expérience utilisateur, proposant une solution comportant deux interfaces pour les utilisateurs finaux et les administrateurs.\n\nInterface Utilisateurs (Widget):\n- L'interface pour les utilisateurs finaux s'intègre facilement à vos applications métier.\n\nInterface Administrateurs (Back Office):\n- Les administrateurs de la solution ont accès à une interface spécifique pour gérer les projets et les interactions.\n- Deux niveaux d'administrateurs existent :\n  1. Les Super Admin qui ont accès à tous les projets au sein de l'organisation.\n  2. Les Admin qui sont les responsables opérationnels d'un ou plusieurs projets.", metadata={'source': 'data/text/WeLoop c_est quoi _.txt'})

### Load Falcon-7B-Instruct:

In [13]:
# flush GPU VRAM before loading the model

gc.collect()
torch.cuda.empty_cache()

In [14]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

model_name = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct
model = AutoModelForCausalLM.from_pretrained(
    model_name, trust_remote_code=True, load_in_8bit=True, device_map="auto"
)
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
# explicitly add mapping for Falcon-7B-instruct in AutoModelForCasualLM

from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES

# Explicitly add the mapping here
MODEL_FOR_CAUSAL_LM_MAPPING_NAMES["RefinedWebModel"] = "RWForCausalLM"

In [16]:
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.max_new_tokens = 512
generation_config.use_cache = True
generation_config.repetition_penalty = 1.5
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 512,
  "pad_token_id": 11,
  "repetition_penalty": 1.5,
  "temperature": 0,
  "transformers_version": "4.30.0"
}

#### First Exmaple:

In [17]:
prompt = """
Tu permettes à un utilisateur poser des questions concernant l'outil Weloop.
Si une resource pertinente existe tu dois être proposé à l'utilisateur sinon 
tu chercheras à comprendre le problème ou la question de l'utilisateur afin 
de formuler un résumé à destination du support client Weloop.
 
Conversation:
 
----------------
{context}

Human: Comment utiliser WeLoop ?
AI:
""".strip()
 
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)
 
with torch.inference_mode():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
    )

In [18]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Tu permettes à un utilisateur poser des questions concernant l'outil Weloop.
Si une resource pertinente existe tu dois être proposé à l'utilisateur sinon 
tu chercheras à comprendre le problème ou la question de l'utilisateur afin 
de formuler un résumé à destination du support client Weloop.
 
Conversation:
 
----------------
{context}

Human: Comment utiliser WeLoop?
AI: WeLoop est un outil de gestion de projets qui permet de centraliser et de suivre les différentes tâches et actions à réaliser. Il s'agit d'un outil de gestion de projets qui permet de centraliser et de suivre les différentes tâches et actions à réaliser.
User 


### Stop model from rambling

To stop the model from generating redundant sentence, we have to set a stopping criteria.

In [19]:
class StopGenerationCriteria(StoppingCriteria):
    # The __init__ method converts the tokens to their corresponding token IDs using the tokenizer and stores them as stop_token_ids.
    def __init__(
        self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]

    #The __call__ method is called during the generation process and takes input IDs as input. It checks if the last few tokens in the input IDs match any of the stop_token_ids, indicating that the model is starting to generate an undesired response.
    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False

Stopping criteria that detects when the LLM, generates new tokens starting with Utilisateur: or Chatbot:.
When such tokens are detected, the generation process will be stopped to prevent undesired outputs.

In [20]:
# stopping criteria

stop_tokens = [["Human", ":"], ["AI", ":"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)]
)

In [21]:
# pipeline that incorporates the stopping criteria and our generation configuration.

generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    stopping_criteria=stopping_criteria,
    generation_config=generation_config,
)
 
llm = HuggingFacePipeline(pipeline=generation_pipeline)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

In [22]:
# pipeline example

res = llm(prompt)
print(res)

 WeLoop est un outil de gestion de projets qui permet de centraliser et de suivre les différentes tâches et actions à réaliser. Il s'agit d'un outil de gestion de projets qui permet de centraliser et de suivre les différentes tâches et actions à réaliser.
User 


### Conversation Chain with Memory

In [23]:
chain = ConversationChain(llm=llm)
print(chain.prompt.template) # default template

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


### Custom Prompt

In [24]:
template = """
Ce qui suit est une conversation entre un humain et une IA. 
L'IA permet à l'utilisateur de poser des questions sur l'outil Weloop. 
Si une ressource pertinente existe, elle doit être proposée à l'utilisateur, 
sinon l'IA va essayer de comprendre le problème ou la question de l'utilisateur 
afin de formuler un résumé pour le support client Weloop.
Si l'IA ne connaît pas la réponse à une question, elle dit sincèrement qu'elle ne sait pas.
qu'il ne sait pas. L'IA répond toujours uniquement en français.
_______

Current conversation:
{history}
Human: {input}
AI:""".strip()
 
prompt = PromptTemplate(input_variables=["history", "input",], template=template)

In [25]:
# remember our previous conversation context while addressing the current question

memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)
 
chain = ConversationChain(llm=llm, memory=memory, prompt=prompt, verbose=True)

In [26]:
# Example

text = "Quelles sont les étapes pour la personnalisation de l'interface utlisateur WeLoop en marque blanche?"
res = chain.predict(input=text)
print(res)



> Entering new ConversationChain chain...
Prompt after formatting:
Ce qui suit est une conversation entre un humain et une IA. 
L'IA permet à l'utilisateur de poser des questions sur l'outil Weloop. 
Si une ressource pertinente existe, elle doit être proposée à l'utilisateur, 
sinon l'IA va essayer de comprendre le problème ou la question de l'utilisateur 
afin de formuler un résumé pour le support client Weloop.
Si l'IA ne connaît pas la réponse à une question, elle dit sincèrement qu'elle ne sait pas.
qu'il ne sait pas. L'IA répond toujours uniquement en français.
_______

Current conversation:

Human: Quelles sont les étapes pour la personnalisation de l'interface utlisateur WeLoop en marque blanche?
AI:

> Finished chain.


1. Analyse des besoins et objectifs du client
2. Création d'une interface utilisateur personnalisée
3. Intégration de l'interface utilisateur personnalisée dans le produit
4. Tests et validation de l'interface utilisateur personnalisée
5. Support et maintenanc

In [27]:
text = "Racontez une histoire à propos de WeLoop"
res = chain(text)
print(res['response'])



> Entering new ConversationChain chain...
Prompt after formatting:
Ce qui suit est une conversation entre un humain et une IA. 
L'IA permet à l'utilisateur de poser des questions sur l'outil Weloop. 
Si une ressource pertinente existe, elle doit être proposée à l'utilisateur, 
sinon l'IA va essayer de comprendre le problème ou la question de l'utilisateur 
afin de formuler un résumé pour le support client Weloop.
Si l'IA ne connaît pas la réponse à une question, elle dit sincèrement qu'elle ne sait pas.
qu'il ne sait pas. L'IA répond toujours uniquement en français.
_______

Current conversation:
Human: Quelles sont les étapes pour la personnalisation de l'interface utlisateur WeLoop en marque blanche?
AI: 

1. Analyse des besoins et objectifs du client
2. Création d'une interface utilisateur personnalisée
3. Intégration de l'interface utilisateur personnalisée dans le produit
4. Tests et validation de l'interface utilisateur personnalisée
5. Support et maintenance de l'interface uti

### Text Decoration

In [28]:
# response dict
res.keys()

dict_keys(['input', 'history', 'response'])

In [29]:
# response text
print(res["response"])

 

WeLoop was founded in 2016 by a group of entrepreneurs who wanted to create a platform that would help businesses and individuals manage their time more efficiently. The idea was born out of a frustration with existing productivity tools, which were clunky and difficult to use. 

The founders recognized that there was a need for a tool that could help people stay on top of their tasks and prioritize their most important work. They set out to design a tool that would be easy to use and intuitive, and they spent a lot of time researching and testing different approaches before finally settling on the current design. 

Since its launch, WeLoop has been used by millions of users around the world to manage their tasks and stay organized. It has become a go-to tool for busy professionals and small business owners alike, and is now one of the leading productivity platforms on the market. 

WeLoop is constantly evolving and improving, and the team is always looking for new ways to make it e

In [30]:
# final prompt template
text = "Quels sont les premièrs messages de WeLoop ?"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
Ce qui suit est une conversation entre un humain et une IA. 
L'IA permet à l'utilisateur de poser des questions sur l'outil Weloop. 
Si une ressource pertinente existe, elle doit être proposée à l'utilisateur, 
sinon l'IA va essayer de comprendre le problème ou la question de l'utilisateur 
afin de formuler un résumé pour le support client Weloop.
Si l'IA ne connaît pas la réponse à une question, elle dit sincèrement qu'elle ne sait pas.
qu'il ne sait pas. L'IA répond toujours uniquement en français.
_______

Current conversation:
Human: Quelles sont les étapes pour la personnalisation de l'interface utlisateur WeLoop en marque blanche?
AI: 

1. Analyse des besoins et objectifs du client
2. Création d'une interface utilisateur personnalisée
3. Intégration de l'interface utilisateur personnalisée dans le produit
4. Tests et validation de l'interface utilisateur personnalisée
5. Support et maintenance de l'interface uti

## Question Answer Chain to show source of the answer:

In [31]:
# create the chain to answer questions

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [32]:
# show current prompt template

print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [33]:
# set context within the prompt template

qa_chain.combine_documents_chain.llm_chain.prompt.template = """"
Tu permettes à un utilisateur poser des questions concernant l'outil Weloop. 
Si une resource pertinente existe tu dois être proposé à l'utilisateur sinon 
tu chercheras à comprendre le problème ou la question de l'utilisateur 
afin de formuler un résumé à destination du support client Weloop.
Tu réponds toujours uniquement en français.
----------------
{context}

Question: {question}
Helpful Answer:"""

In [34]:
# decorate response

def trim_string(input_string):
    input_string = str(input_string)
    trim_index = input_string.find("### Human:")
    if trim_index != -1:  # If the phrase is found
        return input_string[:trim_index]
    else:
        return input_string  # If the phrase isn't found, return the original string

In [35]:
## cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    temp_resp = wrap_text_preserve_newlines(llm_response['result'])
    temp_resp = trim_string(temp_resp)
    print(temp_resp)
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [36]:
# full example
query = "Comment utiliser WeLoop ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Go to the WeLoop Help Center and search for "How to use WeLoop" to find detailed instructions on how to use
the platform.


Sources:
data/text/Créer des groupes utilisateurs.txt
data/text/Créer des groupes utilisateurs.txt
data/text/Créer des groupes utilisateurs.txt
data/text/Créer des groupes utilisateurs.txt


In [37]:
# example of irrelevant question
query = "Quel est l'animal officielle de WeLoop ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


The animal that is the official mascot of Weloop is a panda bear.


Sources:
data/text/WeLoop c_est quoi _.txt
data/text/WeLoop c_est quoi _.txt
data/text/WeLoop c_est quoi _.txt
data/text/WeLoop c_est quoi _.txt


In [38]:
# another example
query = "Quelles sont les étapes pour la personnalisation de l'interface utlisateur WeLoop ?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



1. Accédez à votre espace de travail Weloop.
2. Rendez-vous dans le module "Customize".
3. Configurez l'interface utilisateur en accord avec l'identité visuelle du projet ou de l'entreprise.
4. Créez les messages à diffuser à chaque étape du parcours utilisateur.

Grâce à Weloop, vous pouvez bénéficier d'une personnalisation complète de l'interface utilisateur, vous
permettant ainsi de promouvoir votre propre marque tout en offrant une expérience utilisateur cohérente et
engageante.


Sources:
data/text/Customizer votre widget.txt
data/text/Customizer votre widget.txt
data/text/Customizer votre widget.txt
data/text/Customizer votre widget.txt
